# 일단 서버에 몽고db깔고, 아마존서버 연결하자
- https://drive.google.com/drive/folders/1lx1ugXGKjiYcTZ5FynmQgPL8FM_WAD9v

- boro3T로 연결 성공하면
- open shell하고 이제부터 시작

In [ ]:
#데이터베이스 사용(생성)
use dss

#데이터베이스 목록보기 (단 데이터가 1개라도 있어야 보임)
show dbs

#<database_name><collection_name><Function>({<query>})
#데이터베이스 데이터 삽입하기
db.user.insert({"name":"alice","age":20,"email":"alice@gmail.com"})

#데이터베이스 드랍(삭제)
db.dropDatabase()

#컬렉션 만들기
db.createCollection(name,[option])
# name : 컬렉션의 이름 
# option : 
#     - capped : 최대용량을 설정을 사용
#     - autoIndex : _id라는 column명으로 자동으로 index가 생성
#     - size : 숫자데이터를 넣어서 byte단위를 지정
#     - max : 숫자데이터를 넣어서 document갯수를 지정

# 사용예시) 
db.createCollection("info1", { capped:true, autoIndex:true, size:500, max:5})
db.createCollection("info2", { capped:true, autoIndex:true, size:50, max:5})

#아티클이라는 컬렉션 이름으로 넣어보기 : 이렇게 체이닝으로 컬렉션과 데이터를 한꺼번에 넣을수도 있다.
db.article.insert({"title":"data science", "contents":"mongodb"})

#아티클 삭제
db.article.drop()

# 컬렉션 목록 보기
show collections

# 예제 1. max가 5라서 5개넘으면 예전거 지우면서 들어가고
# db.info1.insert( [
#     { "subject":"python", "level":3 },
#     { "subject":"web", "level":1 },
#     { "subject":"sql", "level":2 },
#     { "subject":"python", "level":3 },
#     { "subject":"web", "level":1 },
#     { "subject":"sql", "level":2 },
# ])

# 예제 2. size가 50이라서 50바이트 넘어가면 예전거 지워진다.
# db.info2.insert( [
#     { "subject":"python", "level":3 },
#     { "subject":"web", "level":1 },
#     { "subject":"sql", "level":2 },
#     { "subject":"python", "level":3 },
#     { "subject":"web", "level":1 },
#     { "subject":"sql", "level":2 },
# ])

# db.info.insert([
#     { "subject":"r", "level":3, "comments": [{"name":"jin","msg":"bad"},
#     {"name":"alice","msg":"hi"}] }, { "subject":"java", "level":1, "comments":
#     [{"name":"po","msg":"check"}] },
#     { "subject":"javascript", "comments": [{"name":"jin","msg":"nice"}] },
#     { "subject":"html", "level":3, "comments": [] },
#     { "subject":"css", "level":1, "comments": [{"name":"alice", "msg":"hello"}] },
#     { "subject":"python", "level":2, "comments": [{"name":"alice", "msg":"hello"}] },
#     { "subject":"python", "level":5, "comments": [{"name":"jin","msg":"nice"}] }
# ])



# find 찾기기능 --> db.<collection_name>.find(query, projection)
    # query : 조건을 작성 (mysql에서 where부분만)
    # projection : 보여지는 컬럼을 정의 (select절 부분만)
    # 예시 : db.info.find({"subject":"python"})

    # 연산자사용해서 찾기 : $lte(작거나 같은), $lt(작은), $gte(크거나 같은), $gt(큰), $in(리스트안에) 
        # db.info.find({"level":{$lte:2}})
        # db.info.find({"subject":{$in:["java","python"]}})

    # 논리연산자 : $or, $and, $not, $nor(not or)
        # db.info.find({$and:[{"subject":"python"},{"level":{$gte:4}}]})

    # $where - 자바그크립트 코드 사용이 가능함!
    # comments의 길이가 1인녀석 찾기
        # db.info.find({$where:"this.comments.length == 1"})

    # $elemMatch - list value데이터 안에 있는 element를 매치시켜서 검색
        # db.info.find({"comments":{$elemMatch :{"name":"alice"}}})

    # $exists - 키값이 존재하는지 안하는지에 따른 조건으로 검색
        # db.info.find({ level : {$exists:true}})

    # projection - 도큐먼트를 조회할때 보여지는 컬럼을 정의
    # 설정을 true로 하던가 false설정 하여 보여지는값을 결정한다( _id 값은 설정을 하지 않으면 true)
        # db.info.find({},{"_id":false, "level":false, "subject":true})
    
    # $slice - document를 읽어올때 출력갯수를 설정할때 사용
        # db.info.find({}, {comments:{$slice:1}})

    # find method - sort, skip. limit
        # db.info.find().sort({level:1}) #오름차순
        # db.info.find().sort({level:-1}) #내림차순

        # db.info.find().skip(2) # 2개 스킵하고 보여준다
        # db.info.find().limit(3) # 3개까지 보여준다
        # db.info.find().skip(2).limit(3) # 2개 스킵한뒤 바로 나오는 3개만 보여준다.

        
#자바함수 쓰기
# var showSkip = function(start){
#     return db.info.find().skip(start);
# }
# showSkip(3)


# quiz
# var showPage = function(page, block){
#     var x = (page-1)*block
#     return db.info.find().skip(x).limit(block);
# }
# showPage(2,3)

In [1]:
# update

    # db.collection_name.update( query, update, {upsert :<bool>, multi:<bool>})

    # query : mysql 에서 where절에 해당되는 구문
    # update : mysql 에서 set절에 해당되는 구문
    # upsert : insert + update 합성어 --> 데이터가 있으면 업데이트, 없으면 insert
    # multi : (default : false) 여러개의 documents를 수정할때 true로 옵션을 설정

    #사용 예제
    # db.info.update(
    #     {subject:"css"},
    #     {subject:"sass", level:2, comment s :[{name:"alice",msg:"hello"}]}
    # )
    
    # db.info.update(
    # { subject:"less" },
    # { subject:"less" , level:3 , comments:[{name:"alice", msg:"hello"}]},
    # { upsert : true }
    # )
    
    #---- $set : 특정필드를 수정할때 사용
    #---- $unset : 특정필드를 제거할 때 사용
    # db.info.update(
    #     {subject :"java"},
    #     {$set:{level:4}},
    #     {multi:true}
    # )
    # level이 3이하 데이터의 레벨을 1로 변경
    # db.info.update(
    #     {level : {$lte:3}},
    #     {$set:{level:1}},
    #     {multi:true}
    # )
    # 값이 없으면 레벨 3을 넣어준다
    # db.info.update(
    #     {level : {$exists:false}},
    #     {$set:{level:3}},
    #     {multi:true}
    # )


    #---- $push : 특정필드의 배열에 엘리먼트를 추가
    #---- $each : 배열에 여러개의 엘리먼트를 추가 할 수 있습니다.
    # subject가 html인 document에 comment를 추가
    # db.info.update(
    #     {subject:"html"},
    #     {$push : {comments:{name:"peter", msg:"html is good"}}}
    # )
    # 여러개 추가
    # db.info.update(
    #     {subject:"html"},
    #     {$push : {comments:{$each :[{name:"jin", msg:"html is good"},{name:"alice", msg:"html is not good"}]}}}
    # )


    #---- $pull : 배열의 값을 제거할때 사용합니다.
    #---- $in : 여러개의 배열값을 선택하여 제거할때 사용합니다.
    # db.info.update(
    #     {subject:"html"},
    #     {$pull : {comments:{$in :[{name:"jin", msg:"html is good"},{name:"alice", msg:"html is not good"}]}}}
    # )

    
    

# pymongo

In [ ]:
# 설치하기
# https://drive.google.com/drive/folders/1lx1ugXGKjiYcTZ5FynmQgPL8FM_WAD9v

In [3]:
import pymongo
import requests, pymongo
import pandas as pd

In [7]:
client = pymongo.MongoClient('mongodb://13.xx.xx.xx:27017/') #내 mongodb서버에로그인
client

In [10]:
# 데이터베이스 선택
db = client.brightkim
db

Database(MongoClient(host=['13.209.5.42:27017'], document_class=dict, tz_aware=False, connect=True), 'brightkim')

In [9]:
# 데이터베이스의 컬렉션 리스트를 확인
db.collection_names()

['info2', 'info1', 'info']

In [11]:
# 컬렉션 선택
collection = db.info
collection

Collection(Database(MongoClient(host=['13.209.5.42:27017'], document_class=dict, tz_aware=False, connect=True), 'brightkim'), 'info')

### Find


In [14]:
# find_one : 한개의 도큐먼트를 가져옵니다.
document = collection.find_one({ "subject":"java"})
document
# 찾아쓰기 참 좋다아~

{'_id': ObjectId('5b5186b8def974a5f238fe5b'),
 'comments': [{'msg': 'check', 'name': 'po'}],
 'level': 4.0,
 'subject': 'java'}

In [16]:
# find : 여러개의 도큐먼트를 가져옵니다.
documents = collection.find({"subject":"java"})
documents
# 아래 output을 보면 오브젝트가 알아서 하나의 커서로 떨어지는걸 알 수 있다.

In [17]:
datas = list(documents)
len(datas)

1

In [18]:
datas

[{'_id': ObjectId('5b5186b8def974a5f238fe5b'),
  'comments': [{'msg': 'check', 'name': 'po'}],
  'level': 4.0,
  'subject': 'java'}]

In [20]:
list(documents) # 이미 앞에서 cursor를 사용했기때문에 또 사용할 수 없다.

[]

In [21]:
# count - documents 의 갯수를 가져오는 함수
documents = collection.find()
documents.count()

8

In [28]:
# sort - 정렬
documents = collection.find({"level":{"$lte":3}}).sort("level", pymongo.DESCENDING)
list(documents)

[{'_id': ObjectId('5b5186b8def974a5f238fe5c'),
  'comments': [{'msg': 'nice', 'name': 'jin'}],
  'level': 3.0,
  'subject': 'javascript'},
 {'_id': ObjectId('5b5186b8def974a5f238fe5d'),
  'comments': [{'msg': 'html is good', 'name': 'peter'}],
  'level': 3.0,
  'subject': 'html'},
 {'_id': ObjectId('5b5186b8def974a5f238fe5a'),
  'comments': [{'msg': 'bad', 'name': 'jin'}, {'msg': 'hi', 'name': 'alice'}],
  'level': 1.0,
  'subject': 'r'},
 {'_id': ObjectId('5b5186b8def974a5f238fe5e'),
  'comments': [{'msg': 'hello', 'name': 'alice'}],
  'level': 1.0,
  'subject': 'css'},
 {'_id': ObjectId('5b5186b8def974a5f238fe5f'),
  'comments': [{'msg': 'hello', 'name': 'alice'}],
  'level': 1.0,
  'subject': 'python'},
 {'_id': ObjectId('5b595a10638351fe9bd415bf'),
  'comments': [{'msg': 'hello', 'name': 'alice'}],
  'level': 1.0,
  'subject': 'less'}]

In [29]:
pymongo.DESCENDING, pymongo.ASCENDING

(-1, 1)

### insert

In [32]:
# insert_one
data = {"subject":"css", "level":1, "comments":[{"name":"peter","msg":"easy"}]}
result = collection.insert_one(data)
result.inserted_id

ObjectId('5b596811429ebb04e4cc4b33')

In [33]:
#insert_many
datas = [
    {"subject":"webpack", "level":2, "comments":[{"name":"peter","msg":"easy"}]},
    {"subject":"gulp", "level":3, "comments":[{"name":"peter","msg":"easy"}]},
    {"subject":"hahaha", "level":4, "comments":[{"name":"peter","msg":"easy"}]},
]
result = collection.insert_many(datas)
result

## 보너스 : 직방데이터 크롤링 후 저장~!

In [ ]:
url = 'https://api.zigbang.com/v3/items?detail=true&item_ids=[12247138,12268011,12269100,12227065,12253070,12150440,12229549,12248873,12274665,12231989,12217749,12087003,11932242,12203883,12251372,12253859,12270993,12206949,12211602,12217708,12254130,12193703,12247672,12247355,12168058,12066173,12230618,12007461,12039940,12271061,12077479,12251672,12234265,12214940,12216385,12269033,12226910,12254133,12153257,11881108,12251797,12243949,12195037,12038915,12239822,12251676,11892233,12229639,12115547,12257609,12137098,12205764,12088126,12268429,12257981,11616166,12259043,12229167,12254380,11997766]'
